
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 非同期計算

:label: `sec_async`

今日のコンピューターは高度な並列システムであり、複数の CPU コア (多くの場合、コアごとに複数のスレッド)、GPU ごとに複数の処理要素、および多くの場合、デバイスごとに複数の GPU で構成されています。つまり、多くの場合、異なるデバイス上で、多くの異なる処理を同時に処理できます。残念ながら、Python は、少なくとも特別な支援がなければ、並列および非同期コードを記述するのに優れた方法ではありません。結局のところ、Python はシングルスレッドであり、これは将来も変わる可能性は低いです。 MXNet や TensorFlow などのディープ ラーニング フレームワークは、パフォーマンスを向上させるために*非同期プログラミング*モデルを採用していますが、PyTorch は Python 独自のスケジューラーを使用しているため、別のパフォーマンス トレードオフが生じます。 PyTorch の場合、デフォルトでは GPU 操作は非同期です。 GPU を使用する関数を呼び出すと、操作は特定のデバイスのキューに入れられますが、必ずしも後になるまで実行されるわけではありません。これにより、CPU や他の GPU での操作を含む、より多くの計算を並行して実行できるようになります。

したがって、非同期プログラミングがどのように機能するかを理解することは、計算要件と相互依存性を積極的に削減することで、より効率的なプログラムを開発するのに役立ちます。これにより、メモリのオーバーヘッドが削減され、プロセッサの使用率が向上します。


In [1]:
import os
import subprocess
import numpy
import torch
from torch import nn
from d2l import torch as d2l


## バックエンド経由の非同期



ウォームアップとして、次のようなおもちゃの問題を考えてみましょう。ランダムな行列を生成し、それを乗算したいと考えています。 NumPy と PyTorch テンソルの両方でこれを実行して、違いを見てみましょう。 PyTorch `tensor` GPU 上で定義されることに注意してください。


In [2]:
# Warmup for GPU computation
device = d2l.try_gpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 2.2001 sec
torch: 0.0057 sec



PyTorch を介したベンチマーク出力は桁違いに高速です。 NumPy のドット積は CPU プロセッサで実行され、PyTorch の行列乗算は GPU で実行されるため、後者の方がはるかに高速であることが期待されます。しかし、大きな時差は、何か別のことが起こっていることを示唆しています。デフォルトでは、PyTorch では GPU 操作は非同期です。 PyTorch が戻る前にすべての計算を強制的に終了すると、以前に何が起こったかがわかります。フロントエンドが制御を Python に返している間、バックエンドによって計算が実行されています。


In [3]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.0039 sec



大まかに言えば、PyTorch には、たとえば Python を介してユーザーと直接対話するためのフロントエンドと、計算を実行するためにシステムによって使用されるバックエンドがあります。 :numref: `fig_frontends`に示すように、ユーザーは Python や C++ などのさまざまなフロントエンド言語で PyTorch プログラムを作成できます。使用されるフロントエンド プログラミング言語に関係なく、PyTorch プログラムの実行は主に C++ 実装のバックエンドで行われます。フロントエンド言語によって発行された操作は、実行のためにバックエンドに渡されます。バックエンドは、キューに入れられたタスクを継続的に収集して実行する独自のスレッドを管理します。これが機能するには、バックエンドが計算グラフ内のさまざまなステップ間の依存関係を追跡できる必要があることに注意してください。したがって、相互に依存する操作を並列化することはできません。



![](../img/frontends.png) :幅: `300px` :ラベル: `fig_frontends`

依存関係グラフをもう少しよく理解するために、別のおもちゃの例を見てみましょう。


In [4]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='cuda:0')


![](http://d2l.ai/_images/asyncgraph.svg) :label: `fig_asyncgraph`

上記のコード スニペットは、 :numref: `fig_asyncgraph`にも示されています。 Python フロントエンド スレッドが最初の 3 つのステートメントのいずれかを実行するたびに、単にタスクをバックエンド キューに返します。最後のステートメントの結果を*出力する*必要がある場合、Python フロントエンド スレッドは、C++ バックエンド スレッドが変数`z`の結果の計算を完了するまで待機します。この設計の利点の 1 つは、Python フロントエンド スレッドが実際の計算を実行する必要がないことです。したがって、Python のパフォーマンスに関係なく、プログラム全体のパフォーマンスにはほとんど影響がありません。 :numref: `fig_threading`フロントエンドとバックエンドがどのように対話するかを示しています。 

![](http://d2l.ai/_images/threading.svg) :label: `fig_threading`

## バリアとブロッカー



## 計算能力の向上



## まとめ
- 深層学習フレームワークは、Python フロントエンドを実行バックエンドから分離する場合があります。これにより、バックエンドへのコマンドの高速な非同期挿入と、それに関連する並列処理が可能になります。
- 非同期により、かなり応答性の高いフロントエンドが実現します。ただし、過剰なメモリ消費につながる可能性があるため、タスク キューを過剰に満たさないように注意してください。フロントエンドとバックエンドのほぼ同期を保つために、ミニバッチごとに同期することをお勧めします。
- チップ ベンダーは、ディープ ラーニングの効率についてより詳細な洞察を得るために、高度なパフォーマンス分析ツールを提供しています。



## 演習


1. このセクションでは、CPU 上で同じ行列乗算演算のベンチマークを実行します。バックエンド経由で非同期を観察できますか?



[ディスカッション](https://discuss.d2l.ai/t/2564)
